# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 1

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [38]:
import numpy as np
import operator
import collections
import pandas
from random import randrange
import matplotlib.pyplot as plt

### Funções úteis durante o decorrer do relatório

#### Calcular mediana

In [25]:
def calc_median(array):
    values = []
    for v in array:
        if v != '?':
            v = int(v)
            values.append(v)
    return np.median(values)

#### Normalizar dados

In [6]:
def normalize(data):
    columns = data.shape[1] - 1 #subtrai 1 para descartar coluna com classes
    for c in range(columns):
        min_v = np.min(data.iloc[:,c])
        max_v = np.max(data.iloc[:,c])
        data.iloc[:,c] = data.iloc[:,c].apply(lambda v : (v - min_v)/(max_v - min_v))
    return data

#### Calcular acurácia

In [8]:
def accuracy_score(prediction,real_values):
    size = len(real_values)
    corrects = 0
    for index in range(size):
        if prediction[index] == real_values[index]:
            corrects += 1
    return corrects/size

#### Calcular acurácia por classe

In [9]:
def accuracy_score_per_class(prediction,real_values,classe):
    size = len(real_values)
    corrects = 0
    total = 0
    for index in range(size):
        if real_values[index] == classe:
            total += 1
        if prediction[index] == real_values[index] == classe:
            corrects += 1
    return corrects/total

#### Calcular matriz de confusão

In [10]:
def confusion_matrix(predict,real_values):
    classes = set(real_values)
    n_predicts = len(predict)
    n_classes = len(classes)
    confusion_m = np.zeros((n_classes,n_classes))
    for cl in classes:
        for index in range(n_predicts):
            if real_values[index] == cl:
                confusion_m[predict[index]-1,real_values[index]-1] += 1
    return confusion_m

#### Separar banco de dados em treino e teste

In [11]:
def train_test_split(data,target,size=0.3):
    test_size = int(len(target)*size)
    numbers = []
    x_train,y_train,x_test,y_test = [],[],[],[]
    while len(numbers) != test_size:
        v = randrange(len(target))
        if v not in numbers:
            numbers.append(v)
            x_test.append(data.iloc[v,:].values)
            y_test.append(target[v])
    for i in range(len(data)):
        if i not in numbers:
            x_train.append(data.iloc[i,:].values)
            y_train.append(target[i])
    return x_train,y_train,x_test,y_test

## Parte 0 : Banco de dados - Demartology

O banco de dados utilizado foi o "Demartology" que possui as seguintes características:

<ul>
<li>O arquivo CSV com os dados possui 35 colunas, sendo que a última coluna representa de forma numérica uma doenção demartológica.</li>
<li>A coluna 33 (band-like infiltrate) apresenta valores inválidos e que precisam ser removidos ou processados.</li>
<li>Temos 366 amostras no banco de dados.</li>
<li>A coluna 34 representa uma doença demartológica de forma numérica(1 à 6), sendo elas :</li>
    <ol>
        <li>psoriasis</li>
        <li>seboreic dermatitis</li>
        <li>lichen planus</li>
        <li>pityriasis rosea</li>
        <li>cronic dermatitis</li>
        <li>pityriasis rubra pilaris</li>
    </ol>
</ul>


In [14]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


## Parte 1 : KNearest Neighbors
### Funcionamento do algoritmo

O algoritmo conhecido como KNN (K-Nearest Neighbors) é um dos muitos algoritmos para classificação, mas que também possui uma versão que pode ser utilizada para regressão, e um dos primeiros a serem apresentados a iniciantes na área de aprendizado de máquina. Seu funcionamento pode ser descrito nas seguintes etapas.

<ol>
<li>Recebe uma amostra que ainda não foi classificada.</li>
<li>Calcula a distância dessa amostra para todas as outras amostras do banco de dados(a métrica utilizada para calcular a distância pode variar com base nos cenários de projeto.).</li>
<li>Filtra as K amostras mais próximas da amostra não classificada ainda.</li>
<li>Verifica a classe mais frequentes dentre as K amostras filtradas no passo 3.</li>
<li>Classifica a amostra do passo 1 como pertencente a classe obtida no passo 4.</li>
</ol>

### Implementacão do classificador "K-Nearest Neighbors (KNN)"

In [3]:
norm_p = lambda x,y,p : abs((x-y))**p
euclidian = lambda list1,list2: np.sqrt(sum(map(norm_p,list1,list2,[2]*len(list1))))
manhatan = lambda list1,list2: sum(map(norm_p,list1,list2,[1]*len(list1)))

metrics = {
    "euclidian" : euclidian,
    "manhatan" : manhatan
}

class KNNClassifier:
    def __init__(self, metric="euclidian", n_neighbors=3):
        if metric not in metrics:
            message = "invalid metric. the acceptable values are :"
            for k in metrics.keys():
                message += " " + k
            raise Exception(message)
        self.n_neighbors = n_neighbors
        self.metric_name = metric
        self.metric_func = metrics[metric]
    def fit(self, x_train,y_train):
        if len(x_train) != len(y_train):
            raise Exception("the size of inputs must be equals")
        self.x_train = x_train
        self.y_train = y_train
    def predict(self,x_test):
        distances = []
        result = []
        for test in x_test:
            for index in range(len(self.x_train)):
                distance = self.metric_func(self.x_train[index],test)
                distances.append((self.y_train[index], distance))
            distances = sorted(distances, key = lambda tup : tup[1])
            classes = collections.Counter(map(lambda x : x[0], distances[:self.n_neighbors]))
            clas = classes.most_common(1)
            result.append(clas[0][0])
            distances.clear()
        return result

## Parte 1.1 : KNearest Neighbors
### Avaliação

#### Leitura da base "demartology"

In [16]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


#### Pré-processamento

Como informado anteriormente, a coluna 33 do banco de dados possui valores inválidos que atrapalham a execução do algoritmo, logo, afim de evitar a remoção da amostras que possuem esse problema, foi adotado a heurística de substituir os valores inválidos,pontos de interrogação,pela mediana da coluna. A mediana foi escolhida por ser menos sensível a outliers se comparada com a média, por exemplo.

In [18]:
median = calc_median(data.iloc[:,33])
data.iloc[:,33] = list(map(lambda value: median if value == '?' else value, data.iloc[:,33]))
data.iloc[:,33] = data.iloc[:,33].astype(np.int64)

Outro fator que pode atrapalhar o desempenho do algoritomo é o fato de que as colunas possuem dimensões diferentes,consequentemente valores muito diferentes. Logo, para solucionar esse problema, o banco de dados foi normalizado fazendo com que os valores fiquem entre 0 e 1 para todas as colunas de atributos, fazendo com que todas as colunas tenham o "mesmo peso".

In [19]:
data = normalize(data)

#### Execução do algoritmo :

<ul>
    <li> Utilizando K = 7 </li>
    <li> Distância Euclidiana </li>
    <li> 80% do banco de dados como dados de TREINAMENTO </li>
    <li> 20% do banco de dados como dados de TESTE </li>
</ul>

#### Acurácia média para 100 rodadas do algoritmo.

In [28]:
results = []
size_tests = 100
KNN = KNNClassifier('euclidian',7)
for index in range(size_tests):
    x_train,y_train,x_test,y_test = train_test_split(features,classes,0.2)
    KNN.fit(x_train,y_train)
    predictions = KNN.predict(x_test)
    results.append((predictions,y_test))

accuracies = []
for r in results:
    accuracies.append(accuracy_score(r[0],r[1]))
print("Acurácia média : {:.2f}%".format(np.mean(accuracies)*100))

Acurácia média : 96.64


#### Acurácias médias de acerto por classe.

In [35]:
classes_all = set(classes)
for c in classes_all:
    score = []
    for r in results:
        score.append(accuracy_score_per_class(predictions,y_test,c))
    print("Classe {:} : {:.2f}%".format(c,np.mean(score)*100))
    score.clear()

Classe 1 : 96.15%
Classe 2 : 100.00%
Classe 3 : 100.00%
Classe 4 : 92.31%
Classe 5 : 100.00%
Classe 6 : 100.00%


#### Matrizes de confusão para o pior e melhor caso dentre as 100 rodadas de treinamente/teste.

In [45]:
acc = accuracy_score(results[0][0],results[0][1])
higher_value = acc
lower_value = acc
lower = results[0]
higher = results[0]
for index in range(1,len(results)):
    accuracy = accuracy_score(results[index][0],results[index][1])
    if accuracy > higher_value:
        higher = results[index]
        higher_value = accuracy
    if accuracy < lower_value:
        lower = results[index]
        lower_value = accuracy

print("Melhor caso - Acurácia: {:.2f}%".format(accuracy_score(higher[0],higher[1])*100))
print(confusion_matrix(higher[0],higher[1]))
print()
print("Pior caso - Acurácia : {:.2f}%".format(accuracy_score(lower[0],lower[1])*100))
print(confusion_matrix(lower[0],lower[1]))

Melhor caso - Acurácia: 100.00%
[[18.  0.  0.  0.  0.  0.]
 [ 0.  9.  0.  0.  0.  0.]
 [ 0.  0. 16.  0.  0.  0.]
 [ 0.  0.  0. 10.  0.  0.]
 [ 0.  0.  0.  0. 14.  0.]
 [ 0.  0.  0.  0.  0.  6.]]

Pior caso - Acurácia : 89.04%
[[24.  0.  0.  0.  0.  0.]
 [ 1. 12.  0.  2.  0.  0.]
 [ 0.  0. 13.  0.  0.  0.]
 [ 0.  5.  0.  5.  0.  0.]
 [ 0.  0.  0.  0.  7.  0.]
 [ 0.  0.  0.  0.  0.  4.]]


## Parte 2 : Classificador Gaussiano Linear
### Funcionamento do algoritmo

## Parte 2 : Classificador Gaussiano QUA
### Funcionamento do algoritmo